In [43]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [44]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

Probamos contactar a utdt.edu.ar

In [45]:
ping("utdt.edu.ar")

Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar


{'dst': 'utdt.edu.ar',
 'response_type': 0,
 'rtt_ms': 12.863874435424805,
 'len': 28,
 'ttl': 55}

Ahora, creamos una función que recibe un set de destinos a contactar. Devuelve un diccionario que contiene:
* Cantidad de paquetes enviados
* Cantidad de paquetes recibidos
* Cantidad de paquetes perdidos
* Porcentaje de paquetes perdidos
* RTT promedio
* RTT mínimo
* RTT máximo
* Desvío estándar del RTT promedio


In [46]:
def ping_multiple_hosts(hosts):

    res = dict()
    sent_packets = len(hosts)
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for host in hosts:
        resultado = ping(host)
        print(resultado)

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido
    
    if received_packets > 0:
        average_rtt = total_rtt / received_packets

    res["hosts"] = hosts
    res["sent_packets"] = sent_packets
    res["received_packets"] = received_packets
    res["lost_packets"] = lost_packets
    res["loss_percentage"] = (lost_packets / sent_packets) 
    res["total_rtt"] = total_rtt
    res["min_rtt"] = min_rtt if min_rtt != float('inf') else None
    res["max_rtt"] = max_rtt if max_rtt != float('-inf') else None
    res["average_rtt"] = average_rtt if received_packets > 0 else None
    res["sd_rtt"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos

In [47]:

destinos = {"mit.edu", "ie.edu", "sydney.edu.au", "cuhk.edu.cn", "alexu.edu.eg"}

ping_multiple_hosts(destinos)

Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
{'dst': 'cuhk.edu.cn', 'response_type': 0, 'rtt_ms': 339.6496772766113, 'len': 28, 'ttl': 112}
Pinging mit.edu...
Echo-reply recibida de mit.edu
{'dst': 'mit.edu', 'response_type': 0, 'rtt_ms': 44.18611526489258, 'len': 28, 'ttl': 50}
Pinging ie.edu...
Echo-reply recibida de ie.edu
{'dst': 'ie.edu', 'response_type': 0, 'rtt_ms': 12.932300567626953, 'len': 28, 'ttl': 246}
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
{'dst': 'sydney.edu.au', 'response_type': 0, 'rtt_ms': 329.12492752075195, 'len': 28, 'ttl': 107}
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
{'dst': 'alexu.edu.eg', 'response_type': None, 'rtt_ms': None, 'len': None, 'ttl': None}


{'hosts': {'alexu.edu.eg',
  'cuhk.edu.cn',
  'ie.edu',
  'mit.edu',
  'sydney.edu.au'},
 'sent_packets': 5,
 'received_packets': 4,
 'lost_packets': 1,
 'loss_percentage': 0.2,
 'total_rtt': 725.8930206298828,
 'min_rtt': 12.932300567626953,
 'max_rtt': 339.6496772766113,
 'average_rtt': 181.4732551574707,
 'sd_rtt': 177.08247515402383}